In [12]:
import json
from glob import glob

output_path = "/home0/murakamih/work/Watch-BBox-In-Out/output"

def read_json(json_path: str) -> list:
    with open(json_path, 'r') as f:
        _data = json.load(f)
    return _data

for dataset in glob(output_path+"/*"):
    for version in glob(dataset+"/*"):
        for json_path in glob(version + "/*.json"):
            data = read_json(json_path)
            # print(data)
            for ped_token, ann in data.items():
                try:
                    if ann['looking'] != 'Object':
                        continue
                    if ann['gto_category'] == 'vehicle.car':
                        # print(data)
                except TypeError as e:
                    print(e)
                    print(data)
            # break
        # break
    # break

{'9d098effc0424029bb6585849d818386': {'bbox': [1252.0251064238696, 468.1979136162275, 1464.6981725918154, 747.3310650596493], 'looking': 'Object', 'gto_bbox': [985.0960233163494, 450.35172260702666, 1600.0, 900.0], 'gto_center': [1292.5480116581748, 675.1758613035133], 'gto_category': 'vehicle.car'}}
{'9b91c31a93da44f1b09fbdb0bbaddae4': {'bbox': [998.655014293729, 442.49355809249033, 1165.7576013730852, 770.8932886226743], 'looking': 'Object', 'gto_bbox': [949.349505149002, 384.8436627896735, 1600.0, 708.4592693322975], 'gto_center': [1274.6747525745009, 546.6514660609855], 'gto_category': 'vehicle.car'}, 'b79365da484d44cbb64264ed44599e00': {'bbox': [1059.0783410783956, 443.76431395990306, 1197.9746749168348, 736.7968194109338], 'looking': 'Object', 'gto_bbox': [601.9118326366608, 396.82728002184535, 1104.9286217090134, 861.6030896629742], 'gto_center': [853.4202271728371, 629.2151848424098], 'gto_category': 'vehicle.car'}}
{'9b91c31a93da44f1b09fbdb0bbaddae4': {'bbox': [998.65501429372

TypeError: list indices must be integers or slices, not str

In [10]:
# 自分自身のBBox内にアノテーション点がある歩行者の抜粋
import os
from glob import glob
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

def read_json(json_path: str) -> list:
    with open(json_path, 'r') as f:
        _data = json.load(f)
    return _data

def watch_self(bbox: list, p: list) -> int:
    if len(p) == 0:
        return 0
    try:
        if bbox[0] <= p[0] and p[0] <= bbox[2] and bbox[1] <= p[1] and p[1] <= bbox[3]:
            return 1
        else:
            return 0
    except IndexError as e:
        print(e)
        print(bbox, p)

def render(dataset_name, version_name, record_name, ped, bbox, ps):
    img_path = glob(f"{img_root}/{dataset_name}/{version_name}/img/{record_name}.*")[0]

    fig = plt.figure(figsize=(16, 9))
    ax = fig.add_subplot(1, 1, 1)

    px = [p[0] for p in ps]
    py = [p[1] for p in ps]

    ax.scatter(px, py, c='red', s=50)
    r = patches.Rectangle(xy=(bbox[0], bbox[1]), width=bbox[2]-bbox[0], height=bbox[3]-bbox[1], ec='yellow', fill=False)
    ax.add_patch(r)
    im = Image.open(img_path)
    ax.imshow(im)

    savepath = f"../output0121/{dataset_name}/{version_name}/{record_name}_{ped}.png"
    savedir = os.path.dirname(savepath)
    os.makedirs(savedir, exist_ok='True')
    plt.savefig(savepath)
    plt.close()

ann_path = "../annotation"
img_root = "/work/murakamih/pedestrian"  # /work/murakamih/pedestrian/nuimages_ped_1017/v1.0-train/img/

for dataset in glob(ann_path + '/*'):
    dataset_name = dataset.split('/')[-1]
    for version in glob(dataset + '/*'):
        version_name = version.split('/')[-1]
        for records in glob(version + '/*.json'):
            record_name = records.split('/')[-1][:-5]
            anns = read_json(records)
            for ped, ann in anns.items():
                bbox = ann['bbox']
                cnt_watch_self = 0
                for p in ann['look']:
                    cnt_watch_self += watch_self(bbox, p)
                if cnt_watch_self == 3:
                    print(records, ped)
                    render(dataset_name, version_name, record_name, ped, bbox, ann['look'])


../annotation/nuscenes_ped/v1.0-trainval/b71a3ed2e4cc4c3b9add0925bcefe743.json 1c61d97d76d14d5c99448d69f0984734
../annotation/nuscenes_ped/v1.0-trainval/d17738b6c30e4b249dae20d65f27a061.json 2aa555fc106d4051ae07ff72285ae62b
../annotation/nuscenes_ped/v1.0-trainval/d17738b6c30e4b249dae20d65f27a061.json 4414192cab3f408daa27ee8261f2d3ac
../annotation/nuscenes_ped/v1.0-trainval/aed63011222d44f191e98adc21c5c8d4.json 23096eb5cf8545c09dff967a03a849e7
../annotation/nuscenes_ped/v1.0-trainval/fff50c470ffb4215b59db47612b12634.json 9d098effc0424029bb6585849d818386
../annotation/nuscenes_ped/v1.0-trainval/30b09ade4d974719a061f40b5d042075.json 9b91c31a93da44f1b09fbdb0bbaddae4
../annotation/nuscenes_ped/v1.0-trainval/974fb362b83b4eceaed5f7eac8de1fb0.json 292bb970b14a4250b83365cb62bb479c
../annotation/nuscenes_ped/v1.0-trainval/d30e87f6586744c8a082e534e32a75f4.json 2e82cd04908449ba953461ece3c0fb74
../annotation/nuscenes_ped/v1.0-trainval/a826c3d05191408fa558b070d2e9f940.json 66deac7713bb4d4dbcd622c92

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


../annotation/nuscenes_ped/v1.0-trainval/69b5e4deccaf45d0ad2e5fcd42658099.json b834f8bf3977472bba953d5eb91169d4
../annotation/nuscenes_ped/v1.0-trainval/919897e923da4f93bd8f4c479493e823.json 956d42440c794f20a8bc002c3a7e3806
../annotation/nuscenes_ped/v1.0-trainval/50cdd8c5711d437aadfbc0d0537c63d1.json 19ba58c7cd9a42079f176a6a520fc901
../annotation/nuscenes_ped/v1.0-trainval/80c0c510baef40b988854cb2062ae896.json ba060f1b27fc4cd6ac796f7ea797fffd
../annotation/nuscenes_ped/v1.0-trainval/ab87ef3300a04da5b003e57b4cacb2da.json 61a09eb0805740f2aef2fccc318566e5
../annotation/nuscenes_ped/v1.0-trainval/a214743471214d9abb96e0636397471c.json 1d730e0ca9d74417af776bcdd9d136a6
../annotation/nuscenes_ped/v1.0-trainval/3d96f1d844af4b379b7f49b8d7c16bf8.json 884bae901bde4ec1aaf046bb1a03fe99
../annotation/nuscenes_ped/v1.0-trainval/4d33b58082194f60afc47f6a8f0062b8.json 4fcd96f31ec54d908e53170321cd0c59
../annotation/nuscenes_ped/v1.0-trainval/5d3b4b55ef90453cbda5327b66654ac2.json 11319c7f17bf4967b7ec4cae1